# Neural Network using tensorflow
for this model whe used some information from "https://machinelearningmastery.com/tensorflow-tutorial-deep-learning-with-tf-keras/"

In [1]:
# import libraries
import tensorflow.keras as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [2]:
# load data
complete_df = pd.read_csv('data/AmesHousingPreprocessed.csv')
complete_df = complete_df.drop('Unnamed: 0', axis=1)
display(complete_df)

,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,...,Year Remod/Add,Full Bath,Year Built,1st Flr SF,Total Bsmt SF,Garage Area,Garage Cars,Gr Liv Area,Overall Qual,SalePrice
0,0,0,0,0,1,0,0,0,0,0,...,-1.155886,-1.019607,-0.368243,1.281557,0.065626,0.258714,0.309465,0.313663,-0.061812,12.278393
1,0,0,0,1,0,0,0,0,0,0,...,-1.107938,-1.019607,-0.335146,-0.675537,-0.384244,1.197200,-1.003952,-1.192617,-0.771063,11.561716
2,0,0,0,0,1,0,0,0,0,0,...,-1.251781,-1.019607,-0.434438,0.439492,0.631372,-0.744814,-1.003952,-0.334434,-0.061812,12.055250
3,0,0,0,0,1,0,0,0,0,0,...,-0.772303,0.794069,-0.103465,2.450663,2.405858,0.230839,0.309465,1.213467,0.647439,12.404924
4,0,0,0,0,1,0,0,0,0,0,...,0.666134,0.794069,0.856357,-0.593133,-0.279728,0.045000,0.309465,0.260151,-0.771063,12.154253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2898,0,0,0,0,1,0,0,0,0,0,...,-0.005137,-1.019607,0.426092,-0.399998,-0.109323,0.537472,0.309465,-0.980548,-0.061812,11.867097
2899,0,0,0,0,1,0,0,0,0,0,...,-0.053085,-1.019607,0.392995,-0.660086,-0.425141,0.054292,0.309465,-1.180725,-0.771063,11.782953
2900,0,0,0,0,1,0,0,0,0,0,...,0.378446,-1.019607,0.690871,-0.484977,-0.316081,-2.194355,-2.317368,-1.045953,-0.771063,11.790557
2901,0,0,0,0,1,0,0,0,0,0,...,-0.436668,-1.019607,0.095119,0.593999,0.767696,-0.252342,0.309465,-0.215517,-0.771063,12.043554


In [3]:
# split data into input and target
target = complete_df['SalePrice']
input_df = complete_df.drop('SalePrice', axis=1)

display(input_df.head())

,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,...,Garage Yr Blt,Year Remod/Add,Full Bath,Year Built,1st Flr SF,Total Bsmt SF,Garage Area,Garage Cars,Gr Liv Area,Overall Qual
0,0,0,0,0,1,0,0,0,0,0,...,-0.730498,-1.155886,-1.019607,-0.368243,1.281557,0.065626,0.258714,0.309465,0.313663,-0.061812
1,0,0,0,1,0,0,0,0,0,0,...,-0.689605,-1.107938,-1.019607,-0.335146,-0.675537,-0.384244,1.197200,-1.003952,-1.192617,-0.771063
2,0,0,0,0,1,0,0,0,0,0,...,-0.812285,-1.251781,-1.019607,-0.434438,0.439492,0.631372,-0.744814,-1.003952,-0.334434,-0.061812
3,0,0,0,0,1,0,0,0,0,0,...,-0.403350,-0.772303,0.794069,-0.103465,2.450663,2.405858,0.230839,0.309465,1.213467,0.647439
4,0,0,0,0,1,0,0,0,0,0,...,0.782561,0.666134,0.794069,0.856357,-0.593133,-0.279728,0.045000,0.309465,0.260151,-0.771063


In [4]:
# convert the pandas dataframes to numpy ndarrays
X_np = input_df.to_numpy()
y_np = target.to_numpy()

# split the data into 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, train_size=0.7, random_state=1265599650)

# find number of features
n_features = input_df.shape[1]

In [16]:
# define metrics
# Root Mean Square Error function from Keras Metrics. 
# source: https://www.tensorflow.org/api_docs/python/tf/keras/metrics/RootMeanSquaredError
m1 = tf.metrics.RootMeanSquaredError()
m2 = 'mean_absolute_percentage_error'

# create layers
model = tf.Sequential([
    Dense(1, input_shape=(n_features,)),
])

# compile model
# use Adam as optimizer because of fast and accurate converging 
# source: https://medium.com/mdr-inc/from-sgd-to-adam-c9fce513c4bb 
model.compile(optimizer='Adam', loss=tf.metrics.mean_squared_error, metrics=[m1, m2])

In [13]:
# fit the model
# choose batch size of 32 after trying 32, 64 and 128
# source: https://datascience.stackexchange.com/questions/18414/are-there-any-rules-for-choosing-the-size-of-a-mini-batch
model.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Epoch 1/100
64/64 [==============================] - 1s 6ms/step - loss: 142.8961 - root_mean_squared_error: 11.9539 - mean_absolute_percentage_error: 99.3589 - val_loss: 135.1594 - val_root_mean_squared_error: 11.6258 - val_mean_absolute_percentage_error: 96.6020
Epoch 2/100
64/64 [==============================] - 0s 3ms/step - loss: 126.8430 - root_mean_squared_error: 11.2625 - mean_absolute_percentage_error: 93.6009 - val_loss: 119.7918 - val_root_mean_squared_error: 10.9449 - val_mean_absolute_percentage_error: 90.9317
Epoch 3/100
64/64 [==============================] - 0s 3ms/step - loss: 112.2025 - root_mean_squared_error: 10.5926 - mean_absolute_percentage_error: 88.0142 - val_loss: 105.8186 - val_root_mean_squared_error: 10.2868 - val_mean_absolute_percentage_error: 85.4447
Epoch 4/100
64/64 [==============================] - 0s 3ms/step - loss: 98.8802 - root_mean_squared_error: 9.9439 - mean_absolute_percentage_error: 82.6036 - val_loss: 93.1097 - val_root_mean_squared_erro

64/64 [==============================] - 0s 3ms/step - loss: 1.0290 - root_mean_squared_error: 1.0144 - mean_absolute_percentage_error: 5.9310 - val_loss: 0.9689 - val_root_mean_squared_error: 0.9843 - val_mean_absolute_percentage_error: 5.7467
Epoch 33/100
64/64 [==============================] - 0s 3ms/step - loss: 0.9557 - root_mean_squared_error: 0.9776 - mean_absolute_percentage_error: 5.7090 - val_loss: 0.9017 - val_root_mean_squared_error: 0.9496 - val_mean_absolute_percentage_error: 5.5439
Epoch 34/100
64/64 [==============================] - 0s 3ms/step - loss: 0.8992 - root_mean_squared_error: 0.9483 - mean_absolute_percentage_error: 5.5690 - val_loss: 0.8490 - val_root_mean_squared_error: 0.9214 - val_mean_absolute_percentage_error: 5.4128
Epoch 35/100
64/64 [==============================] - 0s 3ms/step - loss: 0.8546 - root_mean_squared_error: 0.9244 - mean_absolute_percentage_error: 5.4660 - val_loss: 0.8081 - val_root_mean_squared_error: 0.8990 - val_mean_absolute_percen

Epoch 64/100
64/64 [==============================] - 0s 4ms/step - loss: 0.4915 - root_mean_squared_error: 0.7010 - mean_absolute_percentage_error: 4.4175 - val_loss: 0.4788 - val_root_mean_squared_error: 0.6919 - val_mean_absolute_percentage_error: 4.3552
Epoch 65/100
64/64 [==============================] - 0s 6ms/step - loss: 0.4846 - root_mean_squared_error: 0.6961 - mean_absolute_percentage_error: 4.3872 - val_loss: 0.4731 - val_root_mean_squared_error: 0.6879 - val_mean_absolute_percentage_error: 4.3349
Epoch 66/100
64/64 [==============================] - 0s 5ms/step - loss: 0.4783 - root_mean_squared_error: 0.6916 - mean_absolute_percentage_error: 4.3624 - val_loss: 0.4676 - val_root_mean_squared_error: 0.6838 - val_mean_absolute_percentage_error: 4.3108
Epoch 67/100
64/64 [==============================] - 0s 4ms/step - loss: 0.4722 - root_mean_squared_error: 0.6872 - mean_absolute_percentage_error: 4.3360 - val_loss: 0.4623 - val_root_mean_squared_error: 0.6799 - val_mean_ab

Epoch 96/100
64/64 [==============================] - 0s 4ms/step - loss: 0.3273 - root_mean_squared_error: 0.5721 - mean_absolute_percentage_error: 3.5799 - val_loss: 0.3284 - val_root_mean_squared_error: 0.5730 - val_mean_absolute_percentage_error: 3.6042
Epoch 97/100
64/64 [==============================] - 0s 8ms/step - loss: 0.3224 - root_mean_squared_error: 0.5678 - mean_absolute_percentage_error: 3.5584 - val_loss: 0.3242 - val_root_mean_squared_error: 0.5694 - val_mean_absolute_percentage_error: 3.5756
Epoch 98/100
64/64 [==============================] - 0s 6ms/step - loss: 0.3182 - root_mean_squared_error: 0.5641 - mean_absolute_percentage_error: 3.5275 - val_loss: 0.3201 - val_root_mean_squared_error: 0.5658 - val_mean_absolute_percentage_error: 3.5555
Epoch 99/100
64/64 [==============================] - 0s 5ms/step - loss: 0.3136 - root_mean_squared_error: 0.5600 - mean_absolute_percentage_error: 3.5045 - val_loss: 0.3154 - val_root_mean_squared_error: 0.5616 - val_mean_ab

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 84        
                                                                 
Total params: 84
Trainable params: 84
Non-trainable params: 0
_________________________________________________________________


### Model 2

In [8]:
#### verebeterpunten:
# validation data gebruiken
# k folds
# andere metrics voor aflezen
# adam vs andere andere sgd methods testen
# hidden layers toevoegen
# aantal nodes per layer aanpassen
# batchsize aanpassen (32, 64 of 128)
# aantal epochs 
# onze loss plotten